# Populate Replay Buffer

In [ ]:
# !pip install minerl anyrl

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [1]:
import json
import gym

from collections import deque

import numpy as np

from anyrl.rollouts import replay

import pickle

In [2]:
import sys
sys.path.append("../functions")

from Utils import save_data, load_data
from Buffer import populate_buffer, add_transition
from DataHelper import parse_load_data

TabError: inconsistent use of tabs and spaces in indentation (Buffer.py, line 44)

In [ ]:
replay_buffer = replay.PrioritizedReplayBuffer(500000, alpha=0.4, beta=0.6, epsilon=0.001)

In [3]:
environment = 'Treechop'
folder = 'new-resources'

In [4]:
# file_path = "/content/drive/My Drive/minerl-resources/datasets/parsed_data_treechop.sav"
# combos_path = "/content/drive/My Drive/minerl-resources/combos.sav"
# unique_places_path = "/content/drive/My Drive/minerl-resources/unique_placements.sav"
# unique_actions_path = "/content/drive/My Drive/minerl-resources/unique_actions_treechop.sav"

file_path = "../{0}/parsed_data.sav".format(folder)
combos_path = "../{0}/action_combos_{1}.sav".format(folder, environment.lower())
unique_places_path = "../{0}/unique_placements.sav".format(folder)
unique_actions_path = "../{0}/unique_actions_{1}.sav".format(folder, environment.lower())

In [5]:
combos = load_data(combos_path)
unique_actions = load_data(unique_actions_path)
unique_places = load_data(unique_places_path)
parsed_data = load_data(file_path)

In [ ]:
match_actions(action, combos)

In [8]:
def match_actions(action, combos):
  return [i for i in range(0, len(combos)) if list(combos[i]) == action][0]

In [7]:
a = [0, 0, [-0.30767011642456055, -5.538063049316406], 1, 1, 0, 0, 0, 1]

In [15]:
replay_buffer = populate_buffer(parsed_data, replay_buffer, combos, environment)

Saved processed data!
Populating buffer...
[0, 0, [-0.30767011642456055, -5.538063049316406], 1, 1, 0, 0, 0, 1]


IndexError: list index out of range

In [ ]:
save_data('../{0}/buffer_actions_{1}.sav'.format(folder, environment.lower()), replay_buffer)